<a href="https://colab.research.google.com/github/Offliners/writeup/blob/main/HW7/homework7-5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2021spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1aQoWogAQo_xVJvMQMrGaYiWzuyfO0QyLLAhiMwFyS2w)　Kaggle: [Link](https://www.kaggle.com/c/ml2021-spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Download Dataset

In [1]:
# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

Wed May 19 16:08:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Download link 1
!gdown --id '1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1pOu3FdPdvzielUZyggeD7KDnVy9iW1uC' --output hw7_data.zip

!unzip -o hw7_data.zip

Downloading...
From: https://drive.google.com/uc?id=1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1
To: /content/hw7_data.zip
7.71MB [00:00, 36.1MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          


## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [3]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

     |████████████████████████████████| 2.2MB 27.5MB/s 
     |████████████████████████████████| 901kB 50.9MB/s 
     |████████████████████████████████| 3.3MB 42.5MB/s 


## Import Packages

In [4]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup
from random import randint

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [5]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

     |████████████████████████████████| 51kB 3.1MB/s 


## Load Model and Tokenizer




 

In [6]:
# model = BertForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

# from transformers import AutoTokenizer, AutoModelForQuestionAnswering
  
# tokenizer = AutoTokenizer.from_pretrained("nyust-eb210/braslab-bert-drcd-384")

# model = AutoModelForQuestionAnswering.from_pretrained("nyust-eb210/braslab-bert-drcd-384")

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
  
tokenizer = AutoTokenizer.from_pretrained("wptoux/albert-chinese-large-qa")

model = AutoModelForQuestionAnswering.from_pretrained("wptoux/albert-chinese-large-qa")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

## Read Data

- Training set: 26935 QA pairs
- Dev set: 3523  QA pairs
- Test set: 3492  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [7]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [8]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [19]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 50
        self.max_paragraph_len = 300
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 256

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = (answer_start_token + answer_end_token) // 2
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start

            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 8

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
# temp_set = ConcatDataset([train_set, dev_set])
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [20]:
def evaluate(data, output, index):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
            if answer.find('[UNK]') != -1:
                countUNK = answer.count('[UNK]')
                flag = 0
                print(f'ID {index} Before : {answer}')
                while answer.find('[UNK]') != -1:
                    window_len = end_index - start_index + 10
                    start = 0
                    end = window_len
                    if answer.find('[UNK]') > 1:
                        target1 = answer[answer.find('[UNK]') - 2]
                    else:
                        target1 = -1
                    if len(answer) - (answer.find('[UNK]') + len('[UNK]')) > 0:
                        target2 = answer[answer.find('[UNK]') + len('[UNK]') + 1]
                    else:
                        target2 = -1
                    count = 0
                    for i in range(len(test_paragraphs[test_questions[index]['paragraph_id']])):
                        if end > len(test_paragraphs[test_questions[index]['paragraph_id']]):
                            final = len(test_paragraphs[test_questions[index]['paragraph_id']])
                        else:
                            final = end + i + 1
                        para = test_paragraphs[test_questions[index]['paragraph_id']][start+i: final]
                        if target1 != -1 and target2 != -1:
                            if para.find(target1) != -1 and para.find(target2) != -1:
                                target = para[para.find(target1) + 1: para.find(target2)]
                                print(f'Target : {target}')
                                answer = answer.replace('[UNK]', target)
                                print(f'ID {index} After  : {answer}')
                                count += 1
                        elif target1 != -1 and target2 == -1:
                            if para.find(target1) != -1 and para.find(target1) != len(para) - 1:
                                target = para[para.find(target1) + 1]
                                print(f'Target : {target}')
                                answer = answer.replace('[UNK]', target)
                                print(f'ID {index} After  : {answer}')
                                count += 1
                        elif target1 == -1 and target2 != -1:
                            if para.find(target2) != -1 and para.find(target2) != 0:
                                target = para[para.find(target2) - 1]
                                print(f'Target After: {target}')
                                answer = answer.replace('[UNK]', target)
                                print(f'ID {index} After  : {answer}')
                                count += 1
                        else:
                            answer = answer.replace('[UNK]', para[answer.find('[UNK]')])
                            flag = 1
                            break
                        if count >= countUNK:
                            break
                        if i == len(test_paragraphs[test_questions[index]['paragraph_id']]) - 1:
                            flag = 1
                    if flag == 1:
                      break
                        
    
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')

## Training

In [21]:
num_epoch = 2
validation = True
logging_step = 100
learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * num_epoch
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps=total_steps)

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

accum_iter = 4 

model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0

    for batch_idx, data in enumerate(tqdm(train_loader)):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        with torch.set_grad_enabled(True):
            # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
            # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
            output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

            # Choose the most probable start position / end position
            start_index = torch.argmax(output.start_logits, dim=1)
            end_index = torch.argmax(output.end_logits, dim=1)
            
            # Prediction is correct only if both start_index and end_index are correct
            train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
            train_loss += output.loss
            
            output.loss = output.loss / accum_iter

            if fp16_training:
                accelerator.backward(output.loss)
            else:
                output.loss.backward()
            
            if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(train_loader)):
                optimizer.step()
                optimizer.zero_grad()
            
            step += 1

            ##### TODO: Apply linear learning rate decay #####
            scheduler.step()

            # Print training loss and accuracy over past logging step
            if step % logging_step == 0:
                print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
                train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output, i) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

Start Training ...


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 1 | Step 100 | loss = 0.606, acc = 0.741
Epoch 1 | Step 200 | loss = 0.546, acc = 0.766
Epoch 1 | Step 300 | loss = 0.625, acc = 0.736
Epoch 1 | Step 400 | loss = 0.576, acc = 0.756
Epoch 1 | Step 500 | loss = 0.552, acc = 0.779
Epoch 1 | Step 600 | loss = 0.673, acc = 0.745
Epoch 1 | Step 700 | loss = 0.568, acc = 0.767
Epoch 1 | Step 800 | loss = 0.552, acc = 0.771
Epoch 1 | Step 900 | loss = 0.555, acc = 0.779
Epoch 1 | Step 1000 | loss = 0.527, acc = 0.754
Epoch 1 | Step 1100 | loss = 0.599, acc = 0.762
Epoch 1 | Step 1200 | loss = 0.588, acc = 0.757
Epoch 1 | Step 1300 | loss = 0.521, acc = 0.784
Epoch 1 | Step 1400 | loss = 0.549, acc = 0.767
Epoch 1 | Step 1500 | loss = 0.554, acc = 0.785
Epoch 1 | Step 1600 | loss = 0.546, acc = 0.791
Epoch 1 | Step 1700 | loss = 0.507, acc = 0.796
Epoch 1 | Step 1800 | loss = 0.548, acc = 0.760
Epoch 1 | Step 1900 | loss = 0.475, acc = 0.812
Epoch 1 | Step 2000 | loss = 0.484, acc = 0.800
Epoch 1 | Step 2100 | loss = 0.558, acc = 0.776
E

ID 39 Before : 新 加 坡 在 近 十 五 人 均 [UNK] 都 比 香 港
ID 39 Before : 新 加 坡 在 近 十 五 年 的 人 均 [UNK] 都 是 亞 州 四 小 龍 第 一 位 ， 而 香 港 的 金 融 中 心 地 位 則 比 新 加 坡
ID 305 Before : 大 肚 平 埔 族 拍 布 拉 族 大 肚 王 與 瑯 [UNK] 番 人 的 反 抗
ID 529 Before : 《 阿 [UNK] 婆 吠 陀 》
ID 652 Before : 夾 桑 之 戰 再 次 使 南 軍 損 失 慘 重 ， 正 面 戰 場 戰 事 稍 緩 和 。 中 央 軍 改 為 通 過 談 判 、 反 間 、 襲 擊 後 方 等 方 式 間 接 作 戰 。 擊 敗 平 安 後 ， 燕 軍 南 下 ， 先 後 經 過 順 德 、 廣 平 、 大 名 ， 並 駐 紮 於 大 名 。 諸 郡 縣 望 風 而 降 。 朱 棣 聽 說 齊 黃 被 貶 ， 上 書 和 談 ， 表 示 「 奸 臣 竄 逐 而 其 計 實 行 ， 不 敢 撤 兵 」 。 朱 允 [UNK] 得 書 ， 與 方 孝 孺 討 論 ， 方 孝 孺 表 示 可 以 藉 此 機 會 遣 使 回 報 ， 拖 延 時 間 ， 並 懈 怠 其 軍 心 ； 同 時 令 遼 東 等 軍 隊 攻 其 後 方 ， 以 備 夾 攻 。 於 是 惠 帝 令 大 理 寺 少 卿 薛 品 去 見 朱 棣 ， 傳 詔 並 秘 密 在 軍 中 散 布 相 關 消 息 。 薛 品 見 朱 棣 ， 說 「 朝 廷 言 殿 下 旦 釋 甲 ， 暮 即 旋 師 。 」 朱 棣 表 示 這 連 三 尺 小 兒 也 騙 不 過 。 薛 品 無 言 以 對 。 五 月 初 一 ， 盛 庸 、 吳 傑 、 平 安 等 分 兵 騷 擾 燕 軍 餉 道 。 朱 棣 遣 使 者 進 京 表 示 盛 庸 等 不 肯 罷 兵
ID 716 Before : 姚 [UNK]
ID 811 Before : 蘇 聯 [UNK] - 47 突 擊 步 槍 的 中 方 授 權 製 造 版 本 56 式 自 動 步 槍
ID 826 Before : 1628 年 年 底 ， 當 時 的 福 建 巡 撫 熊 文 燦 再 度 招 安 鄭

Epoch 2 | Step 100 | loss = 0.392, acc = 0.812
Epoch 2 | Step 200 | loss = 0.360, acc = 0.826
Epoch 2 | Step 300 | loss = 0.380, acc = 0.830
Epoch 2 | Step 400 | loss = 0.337, acc = 0.840
Epoch 2 | Step 500 | loss = 0.411, acc = 0.826
Epoch 2 | Step 600 | loss = 0.348, acc = 0.831
Epoch 2 | Step 700 | loss = 0.393, acc = 0.826
Epoch 2 | Step 800 | loss = 0.348, acc = 0.837
Epoch 2 | Step 900 | loss = 0.394, acc = 0.831
Epoch 2 | Step 1000 | loss = 0.353, acc = 0.837
Epoch 2 | Step 1100 | loss = 0.419, acc = 0.832
Epoch 2 | Step 1200 | loss = 0.452, acc = 0.812
Epoch 2 | Step 1300 | loss = 0.352, acc = 0.836
Epoch 2 | Step 1400 | loss = 0.390, acc = 0.837
Epoch 2 | Step 1500 | loss = 0.376, acc = 0.835
Epoch 2 | Step 1600 | loss = 0.380, acc = 0.840
Epoch 2 | Step 1700 | loss = 0.388, acc = 0.824
Epoch 2 | Step 1800 | loss = 0.390, acc = 0.810
Epoch 2 | Step 1900 | loss = 0.371, acc = 0.844
Epoch 2 | Step 2000 | loss = 0.357, acc = 0.857
Epoch 2 | Step 2100 | loss = 0.340, acc = 0.854
E

ID 32 Before : 因 周 王 是 燕 王 同 母 兄 弟 ， 而 朱 允 [UNK] 怕 他 與 燕 王 呵 成 一 氣
ID 39 Before : 新 加 坡 在 近 十 五 人 均 [UNK] 都 比 香 港
ID 305 Before : 大 肚 平 埔 族 拍 布 拉 族 大 肚 王 與 瑯 [UNK] 番 人 的 反 抗
ID 414 Before : [UNK] 大 壩
Target After: 規
ID 414 After  : 規 大 壩
ID 446 Before : 臺 東 縣 外 海 、 太 平 洋 中 的 海 島 ， 原 名 火 燒 島 ， 為 台 灣 第 四 大 附 屬 島 ， 行 政 上 屬 臺 東 縣 綠 島 鄉 。 島 上 居 民 以 清 朝 中 期 從 小 琉 球 、 東 港 移 民 而 來 的 漢 人 為 主 。 綠 島 舊 名 雞 心 嶼 、 青 仔 嶼 與 火 燒 島 等 ， 阿 美 族 稱 為 [UNK] ， 而 達 悟 族 則 稱 之 為 [UNK] 。 西 方 人 有 稱 之 「 [UNK] 島 」 。 火 燒 島 舊 名 眾 說 紛 紜 ， 以 清 嘉 慶 年 間 大 火 焚 燒 島 嶼 之 說 最 為 大 眾 採 信 。 1937 年 設 火 燒 島 庄 ， 屬 台 東 廳 台 東 郡 管 轄 。 戰 後 火 燒 島 設 鄉 ， 歸 屬 於 台 東 縣
Target : 
ID 446 After  : 臺 東 縣 外 海 、 太 平 洋 中 的 海 島 ， 原 名 火 燒 島 ， 為 台 灣 第 四 大 附 屬 島 ， 行 政 上 屬 臺 東 縣 綠 島 鄉 。 島 上 居 民 以 清 朝 中 期 從 小 琉 球 、 東 港 移 民 而 來 的 漢 人 為 主 。 綠 島 舊 名 雞 心 嶼 、 青 仔 嶼 與 火 燒 島 等 ， 阿 美 族 稱 為  ， 而 達 悟 族 則 稱 之 為  。 西 方 人 有 稱 之 「  島 」 。 火 燒 島 舊 名 眾 說 紛 紜 ， 以 清 嘉 慶 年 間 大 火 焚 燒 島 嶼 之 說 最 為 大 眾 採 信 。 1937 年 設 火 燒 島 庄 ， 屬 台 東 廳 台 東 郡 管 轄 。 戰 後 火 燒 島 設 鄉 ， 歸 屬 於 台 東 縣
Targe

In [22]:
test_paragraphs[test_questions[49]['paragraph_id']]

'部份香港政府機關在九龍灣設置總部，包括機電工程署總部大樓、香港輔助警察隊總部，郵政署亦於2013年在該處設立中央郵件中心。附近亦有九龍灣運動場、公園、體育館等公共設施。九龍灣國際展貿中心是香港主要的國際商品展覽會場之一，附設的辦公室亦可作不同產品陳列室之用。現時國際展貿中心仍繼續舉辦不同的國際展覽，亦會不時舉辦演唱會。2007年，自大型購物中心MegaBox開幕後，改變了九龍灣作為工廠、物流倉庫的印象。MegaBox商場高18層，佔地達110萬平方呎，是觀塘區最大型的購物商場，當中的大型商戶有宜家傢俬、AEON、IMAX影院等。區內的寫字樓自2006年起不斷增建，包括企業廣場三期、國際交易中心、Manhattan Place、傲騰廣場及億京中心等，成為東九龍新商業中心。2012年，香港首座零碳建築零碳天地落成，為香港政府宣布《起動九龍東》計劃後，成為九龍東首批標誌性項目之一。九龍灣也有一些住宅區，有兩大私人屋苑，分別是九龍灣站上蓋的德福花園及麗晶花園。'

In [23]:
# Postprocessing

def postprocessing(result, index):
    if result:
        if result[0] == '，' or result[0] == '。':
            print(f'ID {index} Before : {result}')
            result = result[1:]
            print(f'ID {index} After  : {result}')

        if result[0] == '《' and result[-1] != '》' and result.find('》') == -1:
            print(f'ID {index} Before : {result}')
            result = result + '》'
            print(f'ID {index} After  : {result}')
        
        if result[0] != '《' and result[-1] == '》' and result.find('《') == -1:
            print(f'ID {index} Before : {result}')
            result = '《' + result  
            print(f'ID {index} After  : {result}')
        
    return result

## Testing

In [24]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output, i))

result_file = "result.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
                result[i] = postprocessing(result[i], i)
                f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


ID 26 Before : 拉 丁 文 [UNK]
Target : 中
ID 26 After  : 拉 丁 文 中
ID 33 Before : 爪 哇 。 元 代 航 海 家 汪 大 淵 《 島 夷 誌 略 》 記 載 「 石 塘 之 骨 ， 由 潮 州 而 生 ， 迤 [UNK] 如 長 蛇 ， 橫 海 中 ， 越 海 諸 國 。 俗 雲 萬 里 石 塘
Target : 邐
ID 33 After  : 爪 哇 。 元 代 航 海 家 汪 大 淵 《 島 夷 誌 略 》 記 載 「 石 塘 之 骨 ， 由 潮 州 而 生 ， 迤 邐 如 長 蛇 ， 橫 海 中 ， 越 海 諸 國 。 俗 雲 萬 里 石 塘
ID 49 Before : 大 型 購 物 中 心 [UNK] 開 幕 後
Target : MegaBox
ID 49 After  : 大 型 購 物 中 心 MegaBox 開 幕 後
ID 59 Before : 誤 將 「 [UNK] [UNK] 」 的 中 文 翻 譯 為 「 胡 佳 」
ID 84 Before : [UNK] 校 長 [UNK]. [UNK] [UNK]. von [UNK]
Target After: 表
ID 84 After  : 表 校 長 表. 表 表. von 表
Target After: 表
ID 84 After  : 表 校 長 表. 表 表. von 表
Target After: 表
ID 84 After  : 表 校 長 表. 表 表. von 表
Target After: 表
ID 84 After  : 表 校 長 表. 表 表. von 表
Target After: 表
ID 84 After  : 表 校 長 表. 表 表. von 表
ID 250 Before : 溥 [UNK]
Target : 儁
ID 250 After  : 溥 儁
ID 328 Before : 串 聯 混 合 動 力 悍 馬 四 輪 驅 動 越 野 車 [UNK]
Target : X
ID 328 After  : 串 聯 混 合 動 力 悍 馬 四 輪 驅 動 越 野 車 X
ID 332 Before : 目 前 沒 有 觀 察 到 任 何 語 言 純 [UNK] 以 力 道 來 區 分 

In [25]:
from google.colab import files
files.download("result.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>